## Data Science Technology and Systems PG

### Assignment 1 - Part B : Predictive Modelling


#### Import required libraries


In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import ast

#### Load and Inspect Data


In [100]:
# load the data
df = pd.read_csv("./data/zomato_df_final_data.csv")

# print the shape of the data
df.shape

(10500, 17)

In [101]:
# df info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   address        10500 non-null  object 
 1   cost           10154 non-null  float64
 2   cuisine        10500 non-null  object 
 3   lat            10308 non-null  float64
 4   link           10500 non-null  object 
 5   lng            10308 non-null  float64
 6   phone          10500 non-null  object 
 7   rating_number  7184 non-null   float64
 8   rating_text    7184 non-null   object 
 9   subzone        10500 non-null  object 
 10  title          10500 non-null  object 
 11  type           10452 non-null  object 
 12  votes          7184 non-null   float64
 13  groupon        10500 non-null  bool   
 14  color          10500 non-null  object 
 15  cost_2         10154 non-null  float64
 16  cuisine_color  10500 non-null  object 
dtypes: bool(1), float64(6), object(10)
memory usage: 1

In [102]:
# df head
df.head()

,address,cost,cuisine,lat,link,lng,phone,rating_number,rating_text,subzone,title,type,votes,groupon,color,cost_2,cuisine_color
0,"371A Pitt Street, CBD, Sydney",50.0,"['Hot Pot', 'Korean BBQ', 'BBQ', 'Korean']",-33.876059,https://www.zomato.com/sydney/sydney-madang-cbd,151.207605,02 8318 0406,4.0,Very Good,CBD,Sydney Madang,['Casual Dining'],1311.0,False,#e15307,5.243902,#6f706b
1,"Shop 7A, 2 Huntley Street, Alexandria, Sydney",80.0,"['Cafe', 'Coffee and Tea', 'Salad', 'Poké']",-33.910999,https://www.zomato.com/sydney/the-grounds-of-a...,151.193793,02 9699 2225,4.6,Excellent,"The Grounds of Alexandria, Alexandria",The Grounds of Alexandria Cafe,['Café'],3236.0,False,#9c3203,7.560976,#6f706b
2,"Level G, The Darling at the Star, 80 Pyrmont ...",120.0,['Japanese'],-33.867971,https://www.zomato.com/sydney/sokyo-pyrmont,151.195210,1800 700 700,4.9,Excellent,"The Star, Pyrmont",Sokyo,['Fine Dining'],1227.0,False,#7f2704,10.650407,#6f706b
3,"Sydney Opera House, Bennelong Point, Circular...",270.0,['Modern Australian'],-33.856784,https://www.zomato.com/sydney/bennelong-restau...,151.215297,02 9240 8000,4.9,Excellent,Circular Quay,Bennelong Restaurant,"['Fine Dining', 'Bar']",278.0,False,#7f2704,22.235772,#4186f4
4,"20 Campbell Street, Chinatown, Sydney",55.0,"['Thai', 'Salad']",-33.879035,https://www.zomato.com/sydney/chat-thai-chinatown,151.206409,02 8317 4811,4.5,Excellent,Chinatown,Chat Thai,['Casual Dining'],2150.0,False,#a83703,5.630081,#6f706b


#### Data Exploration and feature engineering

In this phase we will do the following:

- remove useless columns
- check and remove duplicate records
- check the missing value count for each columns and handle it


#### Remove columns

As our objective is to predict the restaurant rating, we will remove some of the fields that may not as useful in predicting it. I have identifies the following fields as less useful:

- address (the relevant information can be extracted from subzone)
- cuisines (the cuisines column contains a multiple cuisines, from the data exploration phase we have identified that there isn't any significant influence in rating with the cuisines)
- link
- phone
- rating text (directly related to rating number)
- title
- groupon (variable with negligile variance)
- color
- cuisine_color
- lat
- lng


In [103]:
# select the required columns
df_model = df[
    ["cost", "cost_2", "cuisine", "votes", "type", "subzone", "rating_number"]
]

# df head
df_model.head()

,cost,cost_2,cuisine,votes,type,subzone,rating_number
0,50.0,5.243902,"['Hot Pot', 'Korean BBQ', 'BBQ', 'Korean']",1311.0,['Casual Dining'],CBD,4.0
1,80.0,7.560976,"['Cafe', 'Coffee and Tea', 'Salad', 'Poké']",3236.0,['Café'],"The Grounds of Alexandria, Alexandria",4.6
2,120.0,10.650407,['Japanese'],1227.0,['Fine Dining'],"The Star, Pyrmont",4.9
3,270.0,22.235772,['Modern Australian'],278.0,"['Fine Dining', 'Bar']",Circular Quay,4.9
4,55.0,5.630081,"['Thai', 'Salad']",2150.0,['Casual Dining'],Chinatown,4.5


In [104]:
# shape of the dataframe
df_model.shape

(10500, 7)

In [105]:
# Convert string representation of lists to actual lists
df_model["cuisine"] = df_model["cuisine"].apply(ast.literal_eval)

# Convert string representation of lists to actual lists, for the type column, ignore the error
# replace nan with empty list
df_model["type"] = df_model["type"].fillna("[]")
df_model["type"] = df_model["type"].apply(ast.literal_eval)

C:\Users\ajult\AppData\Local\Temp\ipykernel_24472\555512657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model["cuisine"] = df_model["cuisine"].apply(ast.literal_eval)
C:\Users\ajult\AppData\Local\Temp\ipykernel_24472\555512657.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model["type"] = df_model["type"].fillna("[]")
C:\Users\ajult\AppData\Local\Temp\ipykernel_24472\555512657.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

### Handling missing values in Target variable and `type` variable


In [106]:
# count na values for each column
df_model.isna().sum()

cost              346
cost_2            346
cuisine             0
votes            3316
type                0
subzone             0
rating_number    3316
dtype: int64

In [107]:
# remove columns where rating number is not available
df_model = df_model.dropna(subset=["rating_number"])
df_model = df_model.dropna(subset=["type"])

In [108]:
# shape of column
df_model.shape

(7184, 7)

In [109]:
df_model.isna().sum()

cost             103
cost_2           103
cuisine            0
votes              0
type               0
subzone            0
rating_number      0
dtype: int64

#### Numerical and Categorical Variables


In [110]:
num_features = ["cost", "votes", "cost_2"]
cat_features = ["cuisine", "type", "subzone"]

### Splitting the data into test and train


In [111]:
X = df_model.drop("rating_number", axis=1)
y = df_model["rating_number"]

# shape of X and y
X.shape, y.shape

((7184, 6), (7184,))

In [112]:
# split data into train and test
# 80% train and 20% test
# random state is set to 0
# so that we get the same split every time
# this is important for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Handling missing numerical variables


Imputing missing values in cost and cost_2 columns using median values.


In [113]:
# handling missing values in cost and cost_2
# fill missing values with the median of the column
cost_median = X_train["cost"].median()
cost_2_median = X_train["cost_2"].median()

X_train["cost"] = X_train["cost"].fillna(cost_median)
X_train["cost_2"] = X_train["cost_2"].fillna(cost_2_median)

X_test["cost"] = X_test["cost"].fillna(cost_median)
X_test["cost_2"] = X_test["cost_2"].fillna(cost_2_median)

# check for missing values
X_train.isna().sum()

cost       0
cost_2     0
cuisine    0
votes      0
type       0
subzone    0
dtype: int64

In [114]:
# check for missing values
X_test.isna().sum()

cost       0
cost_2     0
cuisine    0
votes      0
type       0
subzone    0
dtype: int64

### Feature Engineering

`cuisine` : It is list of all cusines served by the restaurant. There are almost 134 unique cuisines. Out of 134 unique cusines in the dataset, we will take the cuisines that are served in less than 100 restaurants as other cuisines.


In [115]:
# list to store all the cusines to obtain unique cuisines
cuisine_list = []
for cuisine in X_train["cuisine"]:
    cuisine_list.extend(cuisine)

# convert the list to a pandas series
cuisine_series = pd.Series(cuisine_list)

# get unique cuisines
cuisine_list = cuisine_series.unique()

# cuisines that are served in less than 100 restaurants
# will be considered as other
cuisine_counts = cuisine_series.value_counts()

# get the cuisines that are served in more than 100 restaurants
top_cuisines = cuisine_counts[cuisine_counts >= 100].index

# get the cuisines that are served in less than 100 restaurants
other_cuisines = cuisine_counts[cuisine_counts < 100].index

# create new column called other_cuisines,
# set the value of the column to 1 if any of the cuisines in the other_cuisines list is present
# in the list in the cuisine column, else set the value to 0
X_train["other_cuisines"] = X_train["cuisine"].apply(
    lambda x: 1 if any(cuisine in other_cuisines for cuisine in x) else 0
)
X_test["other_cuisines"] = X_test["cuisine"].apply(
    lambda x: 1 if any(cuisine in other_cuisines for cuisine in x) else 0
)


# function to add new column for each cuisine
# check each row and if the cuisine is presents in the cuisine list
# then set the value to 1 else 0
def add_cuisine_columns(df, cuisine_list):
    for cuisine in top_cuisines:
        key = cuisine.lower().replace(" ", "_")
        df[key] = df["cuisine"].apply(lambda x: 1 if cuisine in x else 0)
    return df

In [116]:
add_cuisine_columns(X_train, cuisine_list)

X_train.head()

,cost,cost_2,cuisine,votes,type,subzone,other_cuisines,cafe,modern_australian,italian,...,bar_food,sandwich,vegetarian,australian,healthy_food,desserts,korean,coffee_and_tea,mediterranean,bakery
2653,30.0,3.699187,[Cafe],36.0,[Café],Willoughby,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2707,50.0,5.243902,[Thai],54.0,[Casual Dining],Balmain,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3190,40.0,4.471545,"[Vietnamese, Pho]",56.0,[Casual Dining],Hornsby,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3763,70.0,6.788618,[Indian],22.0,[Casual Dining],Cremorne,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2715,40.0,4.471545,"[Poké, Hawaiian, Japanese]",16.0,[],"Greenwood Plaza, North Sydney",1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
add_cuisine_columns(X_test, cuisine_list)
X_test.head()

,cost,cost_2,cuisine,votes,type,subzone,cafe,modern_australian,italian,chinese,...,bar_food,sandwich,vegetarian,australian,healthy_food,desserts,korean,coffee_and_tea,mediterranean,bakery
1667,95.0,8.719512,"[Italian, Pizza]",92.0,[Casual Dining],Willoughby,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1835,30.0,3.699187,"[Bakery, Cafe]",152.0,"[Bakery, Café]",Redfern,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4511,20.0,2.926829,[Mexican],9.0,[Fast Food],"Macquarie Centre, Macquarie Park",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6088,40.0,4.471545,[Cafe],5.0,[Café],Engadine,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
913,75.0,7.174797,"[Seafood, Australian]",36.0,[Casual Dining],Paddington,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [118]:
X_train = X_train.drop("cuisine", axis=1)
X_test = X_test.drop("cuisine", axis=1)

`type`: The type column has 15 unique values. we will do one hot encoding for this column as well.


In [120]:
# let's aggregate the type column, i.e merge all lists in type column into one list
type_list = []
for type in X_train["type"]:
    type_list.extend(type)

# convert the list to a pandas series
type_series = pd.Series(type_list)

# view the data
# type_series.head()

# get unique types
type_list = type_series.unique()

# type counts
type_counts = type_series.value_counts()


# get the types that are highest in number
top_types = type_counts[type_counts >= 100].index

# get the types that are lowest in number
other_types = type_counts[type_counts < 50].index

# create new column called other_types,
# set the value of the column to 1 if any of the types in the other_types list is present
# in the list in the type column, else set the value to 0
X_train["other_types"] = X_train["type"].apply(
    lambda x: 1 if any(type in other_types for type in x) else 0
)
X_test["other_types"] = X_test["type"].apply(
    lambda x: 1 if any(type in other_types for type in x) else 0
)


# function to add new column for each type
# check each row and if the type is presents in the type list
# then set the value to 1 else 0
def add_type_columns(df, type_list):
    for type in top_types:
        key = type.lower().replace(" ", "_")
        df[key] = df["type"].apply(lambda x: 1 if type in x else 0)
    return df

In [122]:
add_type_columns(X_train, type_list)

X_train.head()

,cost,cost_2,votes,type,subzone,other_cuisines,cafe,modern_australian,italian,chinese,...,coffee_and_tea,mediterranean,bakery,other_types,casual_dining,café,bar,fast_food,pub,food_court
2653,30.0,3.699187,36.0,[Café],Willoughby,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2707,50.0,5.243902,54.0,[Casual Dining],Balmain,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3190,40.0,4.471545,56.0,[Casual Dining],Hornsby,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3763,70.0,6.788618,22.0,[Casual Dining],Cremorne,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2715,40.0,4.471545,16.0,[],"Greenwood Plaza, North Sydney",1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
